In [1]:
%autosave 0

Autosave disabled


# 1. TensorFlow Lite

## Using a Keras pre-trained model

In [2]:
import tensorflow as tf
from tensorflow import keras

2023-11-30 17:40:55.043291: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
!python -V

Python 3.9.13


In [4]:
tf.__version__

'2.9.1'

Download model from:  
https://github.com/DataTalksClub/machine-learning-zoomcamp/releases

In [5]:
# !wget https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [6]:
model = keras.models.load_model('clothing-model.h5')

2023-11-30 17:40:57.558944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 17:40:57.565933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 17:40:57.566516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 17:40:57.567434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [7]:
# !wget http://bit.ly/mlbookcamp-pants -O pants.jpg

--2023-11-30 17:41:00--  http://bit.ly/mlbookcamp-pants
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [following]
--2023-11-30 17:41:00--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23048 (23K) [image/jpeg]
Saving to: ‘pants.jpg’

pants.jpg           100%[===================>]  22.51K  --.-KB/s    in 0s      

2023-11-30 17:41:00 (93.9 MB/s) - ‘pants.jp

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [9]:
img = load_img('pants.jpg', target_size=(299, 299))

In [10]:
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [11]:
X.shape

(1, 299, 299, 3)

In [12]:
preds = model.predict(X)

2023-11-30 17:41:01.934623: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-11-30 17:41:02.561296: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 17:41:02.562049: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 17:41:02.562097: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-11-30 17:41:02.562572: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 17:41:02.562662: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1/1 [==============================] - 3s 3s/step


In [13]:
preds

array([[-1.8682917, -4.761245 , -2.3169842, -1.06257  ,  9.887158 ,
        -2.812433 , -3.666283 ,  3.2003632, -2.6023386, -4.835047 ]],
      dtype=float32)

In [14]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [15]:
# The zip function is used to combine elements from iterables.
dict(zip(classes, preds[0]))

{'dress': -1.8682917,
 'hat': -4.761245,
 'longsleeve': -2.3169842,
 'outwear': -1.06257,
 'pants': 9.887158,
 'shirt': -2.812433,
 'shoes': -3.666283,
 'shorts': 3.2003632,
 'skirt': -2.6023386,
 't-shirt': -4.835047}

## Converting a Keras model to TF-Lite

Documentation: https://www.tensorflow.org/lite/models/convert/convert_models#convert_a_savedmodel_recommended

In [16]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory

tflite_model = converter.convert()

# Save the model. Update extension from .h5 to .tflite
with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7o3bjw02/assets


INFO:tensorflow:Assets written to: /tmp/tmp7o3bjw02/assets
2023-11-30 17:41:26.140431: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-11-30 17:41:26.140475: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-11-30 17:41:26.141329: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp7o3bjw02
2023-11-30 17:41:26.168888: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-11-30 17:41:26.168923: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp7o3bjw02
2023-11-30 17:41:26.257307: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-11-30 17:41:26.285687: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-30 17:41:26.830886: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

In [17]:
!ls -lh

total 163M
-rw-r--r-- 1 jovyan jovyan 83M Dec  7  2021 clothing-model.h5
-rw-r--r-- 1 jovyan jovyan 81M Nov 30 17:41 clothing-model.tflite
-rw-r--r-- 1 jovyan jovyan 25K Nov 30 17:41 notebook.ipynb
-rw-r--r-- 1 jovyan jovyan 23K Nov 30 17:41 pants.jpg


## Using a TF-Lite model

In [18]:
import tensorflow.lite as tflite

In [19]:
# Load model
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')

# Allocate memory for the input and output tensors.
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [20]:
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [21]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [23]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8682916,
 'hat': -4.7612457,
 'longsleeve': -2.316979,
 'outwear': -1.0625672,
 'pants': 9.8871565,
 'shirt': -2.8124275,
 'shoes': -3.666287,
 'shorts': 3.2003636,
 'skirt': -2.6023414,
 't-shirt': -4.8350444}

## Removing TF dependency

In [24]:
from PIL import Image

In [25]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

/tmp/ipykernel_8855/2391656872.py:2: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((299, 299), Image.NEAREST)


In [26]:
# Normalize input. preprocess_input scales input pixels between -1 and 1.
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [27]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [28]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [29]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8682916,
 'hat': -4.7612457,
 'longsleeve': -2.316979,
 'outwear': -1.0625672,
 'pants': 9.8871565,
 'shirt': -2.8124275,
 'shoes': -3.666287,
 'shorts': 3.2003636,
 'skirt': -2.6023414,
 't-shirt': -4.8350444}

## Simpler way of doing it

In [ ]:
# !pip install keras-image-helper

In [ ]:
# !pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [1]:
# import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [4]:
url = 'http://bit.ly/mlbookcamp-pants'
# X = preprocessor.from_path('image.jpg')
X = preprocessor.from_url(url)

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [6]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8682916,
 'hat': -4.7612457,
 'longsleeve': -2.316979,
 'outwear': -1.0625672,
 'pants': 9.8871565,
 'shirt': -2.8124275,
 'shoes': -3.666287,
 'shorts': 3.2003636,
 'skirt': -2.6023414,
 't-shirt': -4.8350444}

# 2. AWS Lambda

## Preparing the code for AWS Lamba

Run the following command to convert a Jupyter notebook into a Python script:

```bash
$ jupyter nbconvert --to script 'notebook.ipynb'
```

Edit script and save it as `lambda_function.py`.

Open a terminal and test the `predict` function in the script:

```bash
$ ipython

import lambda_function
lambda_function.predict("http://bit.ly/mlbookcamp-pants")
```

```json
{'dress': -1.8682916,
 'hat': -4.7612457,
 'longsleeve': -2.316979,
 'outwear': -1.0625672,
 'pants': 9.8871565,
 'shirt': -2.8124275,
 'shoes': -3.666287,
 'shorts': 3.2003636,
 'skirt': -2.6023414,
 't-shirt': -4.8350444}
```

Add the Lambda `lambda_handler` function to process events and test the script again:

```bash
$ ipython

import lambda_function
event = { 'url': 'http://bit.ly/mlbookcamp-pants'}
lambda_function.lambda_handler(event, None)
```

## Preparing a Docker image

Create a Dockerfile using the following AWS Lambda base [image](https://gallery.ecr.aws/lambda/python):

```
FROM public.ecr.aws/lambda/python:3.10
```
TensorFlow Lite (`https://google-coral.github.io/py-repo/ tflite_runtime`) requires to be recompiled in the environment where we want to run it. We can use [tflite-aws-lambda](https://github.com/alexeygrigorev/tflite-aws-lambda) which contains compiled binaries for Amazon Linux (a CentOS-based distribution). Add the following command to the Dockerfile:

```
RUN pip install https://github.com/alexeygrigorev/tflite-aws-lambda/blob/main/tflite/tflite_runtime-2.7.0-cp38-cp38-linux_x86_64.whl
```

Build and run a Docker image:

```bash
$ docker build -t clothing-model .
$ docker run -it --rm -p 8080:8080 clothing-model:latest
```

Run the `test.py` script. We expect to get an error since Lambda requires the `lambda_handler()` function to return a serializable output that can be converted into JSON format. To address this error, update `lambda_function.py` with the `tolist()` method, which converts predictions ("preds" are numpy arrays of type float32) into a Python list.

Rebuild the Docker image and run the `test.py` script again.

## Pushing a Docker image to an Amazon ECR repository

Use the AWS CLI to create an Amazon ECR repository:

```bash
$ aws ecr create-repository --repository-name clothing-tflite-images
```

To authenticate your Docker client to your private ECR registry, run the following command:

```bash
$ aws ecr get-login-password --region <region> | docker login --username AWS --password-stdin <aws_account_id>.dkr.ecr.<region>.amazonaws.com
```

The ```aws ecr get-login-password``` command retrieves the password required to authenticate to your Amazon ECR registry using the docker login command, so you can pull and push images. Be sure that `<region>` matches the region in your registry URL. Also, update your `<aws_account_id>`. 

Tag the local image with the Amazon ECR registry:

```bash
$ PREFIX=<aws_account_id>.dkr.ecr.<region>.amazonaws.com/clothing-tflite-images
$ TAG=clothing-model-xception-v1-001
$ REMOTE_URI=${PREFIX}:${TAG}
$ echo ${REMOTE_URI}

$ docker tag clothing-model:latest ${REMOTE_URI}
```

Push the image using the docker push command:
```bash
$ docker push ${REMOTE_URI}
```

## Creating a Lambda function using a container image

Use the AWS console to create a Lambda function from the container image URI (REMOTE_URI). It might be necessary to update the available memory to 1024 MB and the timeout to 30 seconds in the General Configuration tab.

The following event template in JSON format can be used in order to test the Lambda function:

```JSON
{
  "url": "http://bit.ly/mlbookcamp-pants"
}
```

Check AWS Lambda [pricing](https://aws.amazon.com/lambda/pricing/). For instance, the cost for a Lambda function running on an x86 architecture in the us-east-1 region, and using 1024 MB of memory, is $ 0.0000000167 per 1ms.

## Exposing the Lambda function: API Gateway

Follow these steps to create an API Gateway on the AWS console:

* Create a REST API named `clothing-classification`.
* Add a new  resource called `predict`.
* Create a new HTTP method (`POST`) and integrate it with the Lambda function.
* Test the API gateway using the event template from the previous section.
* Deploy the API assigning the stage name `test`. This will generate the invoke URL `https://<api_id>.execute-api.us-east-1.amazonaws.com/test`.
* Update the `test.py` script with the API endpoint `<invoke URL>/predict` and run it to test the deployment.

If successfull, it will return the following prediction:

```JSON
{'dress': -1.879865288734436,
 'hat': -4.756308555603027,
 'longsleeve': -2.359532594680786,
 'outwear': -1.0892627239227295,
 'pants': 9.903782844543457,
 'shirt': -2.8261797428131104,
 'shoes': -3.6483089923858643,
 'shorts': 3.2411575317382812,
 'skirt': -2.612095355987549,
 't-shirt': -4.852032661437988}
```